In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class optStruct:
    def __init__(self, dataMat, labelMat, C, toler):
        self.dataMat = dataMat
        self.labelMat = labelMat
        self.C  = C
        self.tol = toler
        self.r = dataMat.shape[0]
        self.alpha = np.mat(np.zeros((self.r, 1)))
        self.b = 0
        self.eCache = np.mat(np.zeros((self.r, 2)))

In [3]:
"""
核函数定义
"""
def kernel(x1, x2):
    return x1 * x2.T

In [22]:
def calcEk(k,oS):
#     print('calc')
#     print(oS)
    f_ = float(np.multiply(oS.alpha,oS.labelMat).T * (oS.dataMat * oS.dataMat[k,:].T) + oS.b)
    Ek = f_ - float(oS.labelMat[k])
    return Ek

In [5]:
def updateCache(k, oS):
    Ek = calcEk(k, oS)
    oS.eCache[k] = [1,Ek]

In [6]:
def selectJrand(i,m):
    j=i #we want to select any J not equal to i
    while (j==i):
        j = int(np.random.uniform(0,m))
    return j

In [21]:
def selectI1(i, oS, Ei):
    maxK = -1; maxDeltaE = 0; Ej = 0
    oS.eCache[i] = [1,Ei]  #set valid #choose the alpha that gives the maximum delta E
    validEcacheList = np.nonzero(oS.eCache[:,0].A)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:   #loop through valid Ecache values and find the one that maximizes delta E
            if k == i: continue #don't calc for i, waste of time
            Ek = calcEk(k, oS)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k; maxDeltaE = deltaE; Ej = Ek
        return maxK, Ej
    else:   #in this case (first time around) we don't have any valid eCache values
        j = selectJrand(i, oS.r)
#         print('s')
#         print(oS)
        Ej = calcEk(j, oS)
    return j, Ej 

In [17]:
"""
input:
    i1, i2为两个实例
output:
    
"""
def takeStep(i1, i2, E1, E2, oS):
    if i1 == i2: return 0
    data = oS.dataMat
    label = oS.labelMat
    alpha = oS.alpha
    b = oS.b
    C = oS.C
    alpha1 = alpha[i1]
    alpha2 = alpha[i2]
    x1 = data[i1]
    x2 = data[i2]
    y1 = label[i1]
    y2 = label[i2]
    s = y1 * y2
#     y1_ = float(np.multiply(alpha, label).T * (data * x1.T)) + b
#     y2_ = float(np.multiply(alpha, label).T * (data * x2.T)) + b
#     E1 = y1_ - y1
#     E2 = y2_ - y2
    if y1 != y2:
        L = max(0, alpha2 - alpha1)
        H = min(C, C + alpha2 - alpha1)
    else:
        
        L = max(0, alpha2 + alpha1 - C)
        H = min(C, alpha2 + alpha1)
    if L == H :
        print('L == H')
        return 0
    eta = kernel(x1, x1) + kernel(x2, x2) - 2 * kernel(x1, x2)
    
    if eta > 0:
        alpha2_new = alpha2 + y2 * (E1 - E2) / eta
        if alpha2_new >= H:
            alpha2_new = H
        elif alpha2_new <= L:
            alpha2_new = L
        else:
            alpha2_new = alpha2_new
        
#         alpha1_new = alpha1 + s * (alpha2 - alpha2_new_c)
    else:
        print('eta <= 0')
        return 0
#         f1 = y1 * (E1 + b) - alpha1 * kernel(x1, x1) - \
#              s * alpha2 * kernel(x1, x2)
#         f2 = y2 * (E2 + b) - s * alpha1 * kernel(x1, x2) - \
#              alpha2 * kernel(x2, x2)
#         L1 = alpha1 + s * (alpha2 - L)
#         H1 = alpha1 + s * (alpha2 - H)
#         fiL = L1 * f1 + L * f2 + (1/2) * np.power(L1, 2) * kernel(x1, x1) + \
#               (1/2) * np.power(L, 2) * kernel(x2, x2) + s * L * L1 * kernel(x1, x2)
#         fiH = H1 * f1 + H * f2 + (1/2) * np.power(H1, 2) * kernel(x1, x1) + \
#               (1/2) * np.power(H, 2) * kernel(x2, x2) + s * H * H1 * kernel(x1, x2)
#         eps = 0.001
#         if fiL < fiH - eps:
#             alpha2_new = L
#         elif fiL > fiH + eps:
#             alpha2_new = H
#         else:
#             alpha2_new = alpha2
    alpha1_new = alpha1 + s * (alpha2 - alpha2_new)
    oS.alpha[i1] = alpha1_new
    oS.alpha[i2] = alpha2_new
    updateCache(i1, oS)
    updateCache(i2, oS)
    b1 = E1 + y1 * (alpha1_new - alpha1) * kernel(x1, x1) + y2 * (alpha2_new - alpha2) * kernel(x1, x2) + b
    b2 = E2 + y1 * (alpha1_new - alpha1) * kernel(x1, x2) + y2 * (alpha2_new - alpha2) * kernel(x2, x2) + b
    if alpha1 > 0 and alpha1 < C:
        oS.b = b1
    elif alpha2 > 0 and alpha2 < C:
        oS.b = b2
    else : oS.b = (b1 + b2) / 2
    return 1

In [19]:
def examineExample(i2, oS):
    data = oS.dataMat
    label = oS.labelMat
    alpha = oS.alpha
    b = oS.b
    C = oS.C
    y2 = label[i2]
    alpha2 = alpha[i2]
    E2 = calcEk(i2, oS)
    r2 = E2 * y2
    if((r2 < -oS.tol) and (alpha2 < C) or (r2 > oS.tol and alpha2 > 0)):
        notBound = np.nonzero((alpha.A > 0) * (alpha.A < C))[0] 
#         if(len(notBound) > 1):
#             i1, E1 = selectI1(i2, oS, E2)
#             if takeStep(i1, i2):
#                 return 1
        i1, E1 = selectI1(i2, oS, E2)
        if takeStep(i1, i2, E1, E2, oS):
            return 1
        else :
            return 0
    else:
        return 0

In [24]:
def main(oS):
    numChanged = 0;allSet = True
    
    while(numChanged > 0 or allSet):
        numChanged = 0
        if allSet:
            for i in range(oS.r):
                numChanged += examineExample(i, oS)
        else:
            notBoundIs = np.nonzero((oS.alpha.A > 0) * (oS.alpha.A < oS.C))[0]
            for i in notBoundIs:
                numChanged += examineExample(i,oS)
        if allSet == True:
            allSet = False
        elif numChanged == 0 :
            allSet = True           

In [11]:
df = pd.read_csv('testSet.txt', sep = '\t', names =['x','y', 'label'])

In [12]:
dataMat = np.mat(df[['x', 'y']].values)
labelMat = np.mat(df['label'].values).T

In [13]:
oS = optStruct(dataMat, labelMat, 0.6, 0.001)

In [ ]:
main(oS)

L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H


D:\Programfiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: overflow encountered in add


In [ ]:
oS.alpha